# Saving Algerian Constitution into SurrealDB

## 1. Importing libraries

In [45]:
%pip install python-dotenv==1.0.1 surrealdb==1.0.3 openai==1.63.0

Note: you may need to restart the kernel to use updated packages.


In [46]:
from datetime import datetime
from dotenv import load_dotenv
from surrealdb import Surreal
from openai import OpenAI
import json
import os

load_dotenv(override=True)

True

## 2. Load the data

In [47]:
json_file_path = os.path.join("json", "dz_consti_2020.json")
with open(json_file_path, "r", encoding="utf-8") as f:
    constitution_data = json.load(f)

## 3. Connect to the database

In [48]:
db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [49]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"

def save_constitution():
    for article in constitution_data:
        article_text = article['article']
        title_header = article['title']
        chapter_header = article['chapter']
        # more about this on https://platform.openai.com/docs/guides/embeddings#how-to-get-embeddings
        response = client.embeddings.create(
            input=f"{title_header}: {chapter_header}: {article}",
            model=embed_model_name,
        )
        vector = response.data[0].embedding

        db.insert(
            "articles",
            [{
                "text": article_text,
                "embed_vector": vector,
                "chapter": chapter_header,
                "title": title_header,
                "edition": datetime(2020, 11, 1).strftime('%Y-%m-%dT%H:%M:%SZ'),
            }]
        )


save_constitution()

## 5. Test Search in Database

In [ ]:
# user_question = "worshiping in Algeria"
user_question = "Qu'est-ce que je peux faire dans la guerre?"

response = client.embeddings.create(
    input=user_question,
    model=embed_model_name,
)
q_vector = response.data[0].embedding

results = db.query("""
    SELECT
        text,
        vector::similarity::cosine(embed_vector, $q_vector) AS score
    FROM articles
    ORDER BY score DESC
    LIMIT 10
    """,
    {"q_vector": q_vector},
)

print(json.dumps(results, indent=2, ensure_ascii=False))

[
  {
    "score": 0.17552665383723876,
    "text": "المادة 110 : يُوقَف العمل بالدّستور مدّة حالة الحرب ويتولّى رئيس الجمهوريّة جميع السّلطات. وإذا انتهت المدّة الرّئاسيّة لرئيس الجمهوريّة تمدّد وجوبا إلى غاية نهاية الحرب. في حالة استقالة رئيس الجمهوريّة أو وفاته أو حدوث أيّ مانع آخر له، يخوّل رئيس مجلس الأمّة باعتباره رئيسا للدّولة، كلّ الصّلاحيّات الّتي تستوجبها حالة الحرب، حسب الشّروط نفسها الّتي تسري على رئيس الجمهوريّة. في حالة اقتران شغور رئاسة الجمهوريّة ورئاسة مجلس الأمّة، يتولّى رئيس المجلس الدّستوريّ وظائف رئيس الدّولة حسب الشّروط المبيّنة سابقا."
  },
  {
    "score": 0.16457101500841095,
    "text": "المادة 28 : تنتظم الطّاقة الدّفاعيّة للأمّة، ودعمها، وتطويرها، حول الجيش الوطنيّ الشّعبيّ. تتمثّل المهمّة الدّائمة للجيش الوطنيّ الشّعبيّ في المحافظة على الاستقلال الوطنيّ، والدّفاع عن السّيادة الوطنيّة. كما يضطلع بالدّفاع عن وحدة البلاد، وسلامتها التّرابيّة، وحماية مجالها البرّيّ والجوّيّ، ومختلف مناطق أملاكها البحريّة."
  },
  {
    "score": 0.15715455621030833,
    "text": 